# Data Prep

In this notebook we:
- Load the data
- Check for missing values
- Check for duplicates
- Clean the data
- Split the data into train, validation and test sets

## Summary

#### Strategies for splitting the data

Even though most of the columns are floats, we have some columns with low cardinality. This can be a problem when splitting the data into train, validation and test sets. If we split the data randomly, we can end up with records with the same value in the low cardinality columns in the training and validation/test sets. This can lead to data leakage and the model will not generalize well to unseen data i.e. the model will learn how to predict for only the small subset of values in the low cardinality columns and will not be able to generalize to unseen values.

To avoid this problem, we will use the following strategies to split the data:

1. **GroupKFold**: We will split the data into 5 groups based on the the low cardinallity columns. We will use 4 groups for training and with the remaining group we will split it into validation and test sets. This way, we can make sure that the model generalizes well to unseen data. The proportion of the data in each group is approximately 80% for training and 10% for validation and test sets.
2. **LeaveOneGroupOut**: We will use this strategy to evaluate the model's performance on unseen targets (for the low cardinality targets `potenciaGeradaTG1_2`, `consumoEspecificoTG1_2`). We will leave two non zero groups out for validation and test sets.
3. **RandomSplit**: We will use this strategy to split the data into training, validation and test sets only for comparison purposes.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold

In [2]:
df = pd.read_json('./data/raw/SimulationResult2.json')
df.head()

,step,vazaoVapor,pressaoVapor,temperaturaVapor,cargaVaporTG1,cargaVaporTG2,habilitaTG1,habilitaTG2,potenciaGeradaTG1_2,potenciaGeradaTG2_2,potenciaGeradaTG2_1,potenciaGeradaTG1_1,vazaoVaporEscape,temperaturaVaporEscape,pressaoVaporEscape,consumoEspecificoTG2_2,consumoEspecificoTG2_1,consumoEspecificoTG1_2,consumoEspecificoTG1_1,status
0,0,263.078370,57.419117,728.414570,108.616210,50.721489,0,0,0.0,0.000000,0.000000,0.0,291.986802,403.15,2.3,0.000000,0.000000,0.0,0.0,OK
1,1,282.103961,58.607302,737.854026,102.122037,50.496216,0,0,0.0,0.000000,0.000000,0.0,318.504931,403.15,2.3,0.000000,0.000000,0.0,0.0,OK
2,2,261.112878,58.973315,693.887523,103.789047,51.843660,0,0,0.0,0.000000,0.000000,0.0,279.373804,403.15,2.3,0.000000,0.000000,0.0,0.0,OK
3,3,269.834454,54.318838,748.630084,103.771199,52.995014,0,0,0.0,0.000000,0.000000,0.0,303.054202,403.15,2.3,0.000000,0.000000,0.0,0.0,OK
4,4,283.190729,54.914962,701.431269,111.697638,50.375678,0,1,0.0,2.537824,6.252941,0.0,306.899172,403.15,2.3,11.969184,8.056317,0.0,0.0,OK


As we discussed in the Exploratory Data Analysis, we are going to:
- drop the lines with Fail Simulation Status (`status` == "Falha na simulação")
- Drop the lines with ``potenciaGeradaTG2_2`` < 0 (because this column represent the power generated by the turbine TG2_2, and it can't be negative)
- Drop the targets ``temperaturaVaporEscape`` and ``pressaoVaporEscape`` (because they are constant values and only hinder the model from learning)
- Create a GroupKFold train-test split, as we learned that many columns of the dataset have low cardinality, and we want to make sure that the model can generalize well to unseen data (i.e., we want to test the model on groups it hasn't seen in training)

In [3]:
features = ['vazaoVapor', 'pressaoVapor', 'temperaturaVapor',
            'cargaVaporTG1', 'cargaVaporTG2', 'habilitaTG1', 'habilitaTG2']

In [4]:
targets = ['consumoEspecificoTG1_1', 'consumoEspecificoTG1_2',
           'consumoEspecificoTG2_1', 'consumoEspecificoTG2_2',
           'potenciaGeradaTG1_1', 'potenciaGeradaTG1_2',
           'potenciaGeradaTG2_1', 'potenciaGeradaTG2_2',
           'vazaoVaporEscape']

In [5]:
boolean_columns = ['habilitaTG1', 'habilitaTG2']

In [6]:
dataset = (df
        .query('status == "OK" and potenciaGeradaTG2_2 >= 0')
        .drop(columns=['status', 'step','temperaturaVaporEscape', 'pressaoVaporEscape']))

In [7]:
dataset.to_csv('./data/processed/dataset.csv', index=False)

In [8]:
# Check for duplicates
dataset.duplicated().sum()

0

In [9]:
# Cardinality of each column
dataset.nunique()

vazaoVapor                287936
pressaoVapor              287936
temperaturaVapor          287936
cargaVaporTG1             287936
cargaVaporTG2             287936
habilitaTG1                    2
habilitaTG2                    2
potenciaGeradaTG1_2       139024
potenciaGeradaTG2_2       140644
potenciaGeradaTG2_1       141757
potenciaGeradaTG1_1       143677
vazaoVaporEscape          287771
consumoEspecificoTG2_2    115259
consumoEspecificoTG2_1    139651
consumoEspecificoTG1_2    128923
consumoEspecificoTG1_1    143237
dtype: int64

In [10]:
# Check the balance of the boolean columns
dataset[boolean_columns].value_counts(normalize=True)

habilitaTG1  habilitaTG2
0            0              0.253098
1            0              0.253053
0            1              0.246947
1            1              0.246902
Name: proportion, dtype: float64

In [11]:
dataset.head()

,vazaoVapor,pressaoVapor,temperaturaVapor,cargaVaporTG1,cargaVaporTG2,habilitaTG1,habilitaTG2,potenciaGeradaTG1_2,potenciaGeradaTG2_2,potenciaGeradaTG2_1,potenciaGeradaTG1_1,vazaoVaporEscape,consumoEspecificoTG2_2,consumoEspecificoTG2_1,consumoEspecificoTG1_2,consumoEspecificoTG1_1
0,263.078370,57.419117,728.414570,108.616210,50.721489,0,0,0.0,0.000000,0.000000,0.0,291.986802,0.000000,0.000000,0.0,0.0
1,282.103961,58.607302,737.854026,102.122037,50.496216,0,0,0.0,0.000000,0.000000,0.0,318.504931,0.000000,0.000000,0.0,0.0
2,261.112878,58.973315,693.887523,103.789047,51.843660,0,0,0.0,0.000000,0.000000,0.0,279.373804,0.000000,0.000000,0.0,0.0
3,269.834454,54.318838,748.630084,103.771199,52.995014,0,0,0.0,0.000000,0.000000,0.0,303.054202,0.000000,0.000000,0.0,0.0
4,283.190729,54.914962,701.431269,111.697638,50.375678,0,1,0.0,2.537824,6.252941,0.0,306.899172,11.969184,8.056317,0.0,0.0


In [12]:
low_cardinality_columns = [col for col in dataset.columns if dataset[col].nunique() < 15 and col not in boolean_columns]

print(f'{len(low_cardinality_columns)} columns have low cardinality')
print(f'Percentage of low cardinality unique groups: {len(low_cardinality_columns) / len(dataset.columns) * 100:.2f}%')

0 columns have low cardinality
Percentage of low cardinality unique groups: 0.00%


In [13]:
low_card_desc = (dataset[low_cardinality_columns]
 .nunique()
 .reset_index()
 .rename(columns={ 'index': 'column', 0: 'nunique' }))
low_card_desc['type'] = low_card_desc['column'].map(lambda x: 'feature' if x in features else 'target')
low_card_desc = low_card_desc[['column', 'type', 'nunique']]
low_card_desc

,column,type,nunique


In [14]:
X, y = dataset[features], dataset[targets]

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# Train set and Test set split by random sampling
X_train, X_test_full, y_train, y_test_full = train_test_split(X, y, test_size=0.1, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_full, y_test_full, test_size=0.5, random_state=42)

assert len(X_train) + len(X_test) + len(X_val) == len(X)
assert len(y_train) + len(y_test) + len(y_val)== len(y)
assert len(X) == len(y)
assert len(X) == len(dataset)

print(f'Train: {len(X_train):,} ({len(X_train)/len(dataset):.1%}) samples')
print(f'Test: {len(X_test):,} ({len(X_test)/len(dataset):.1%}) samples')
print(f'Validation: {len(X_val):,} ({len(X_val)/len(dataset):.1%}) samples')

X_train.to_csv('./data/processed/X_train.csv', index=False)
X_test.to_csv('./data/processed/X_test.csv', index=False)
X_val.to_csv('./data/processed/X_val.csv', index=False)
y_train.to_csv('./data/processed/y_train.csv', index=False)
y_test.to_csv('./data/processed/y_test.csv', index=False)
y_val.to_csv('./data/processed/y_val.csv', index=False)

Train: 259,142 (90.0%) samples
Test: 14,397 (5.0%) samples
Validation: 14,397 (5.0%) samples
